In [1]:
import cv2
import time
from datetime import datetime

# Open the webcam (0 = default camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break

        # Current date & time
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Put timestamp text on the image
        cv2.putText(frame, timestamp, (10, frame.shape[0] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

        # Save image with timestamp in filename
        filename_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        filename = f"photo_{filename_time}.jpg"
        cv2.imwrite(filename, frame)
        print(f"Saved {filename}")

        time.sleep(10)  # wait 5 seconds

except KeyboardInterrupt:
    print("Stopped by user.")

cap.release()
cv2.destroyAllWindows()

Saved photo_2025-08-27_10-13-05.jpg
Saved photo_2025-08-27_10-13-15.jpg
Saved photo_2025-08-27_10-13-25.jpg
Saved photo_2025-08-27_10-13-35.jpg
Saved photo_2025-08-27_10-13-45.jpg
Saved photo_2025-08-27_10-13-55.jpg
Saved photo_2025-08-27_10-14-05.jpg
Saved photo_2025-08-27_10-14-16.jpg
Saved photo_2025-08-27_10-14-26.jpg
Saved photo_2025-08-27_10-14-36.jpg
Saved photo_2025-08-27_10-14-46.jpg
Saved photo_2025-08-27_10-14-56.jpg
Saved photo_2025-08-27_10-15-06.jpg
Saved photo_2025-08-27_10-15-16.jpg
Saved photo_2025-08-27_10-15-26.jpg
Saved photo_2025-08-27_10-15-36.jpg
Saved photo_2025-08-27_10-15-46.jpg
Saved photo_2025-08-27_10-15-56.jpg
Saved photo_2025-08-27_10-16-06.jpg
Saved photo_2025-08-27_10-16-16.jpg
Saved photo_2025-08-27_10-16-26.jpg
Saved photo_2025-08-27_10-16-36.jpg
Saved photo_2025-08-27_10-16-46.jpg
Saved photo_2025-08-27_10-16-56.jpg
Saved photo_2025-08-27_10-17-06.jpg
Saved photo_2025-08-27_10-17-16.jpg
Saved photo_2025-08-27_10-17-26.jpg
Saved photo_2025-08-27_10-17

In [5]:
import cv2
import os

# Folder with your saved photos
image_folder = "AR-273"

# Output video filename
video_name = "stopmotion1.avi"

# Get list of images sorted by name (important for correct order!)
images = sorted([img for img in os.listdir(image_folder) if img.endswith(".jpg")])

# Read first image to get size
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# Define video writer (XVID = widely supported codec)
fps = 5  # frames per second (increase for faster playback)
fourcc = cv2.VideoWriter_fourcc(*"XVID")
video = cv2.VideoWriter(video_name, fourcc, fps, (width, height))

for image in images:
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)
    video.write(frame)

video.release()
print(f"Stop-motion video saved as {video_name}")


Stop-motion video saved as stopmotion1.avi
